In [1]:
IMG_SIZE = 80

## Creating the Convolutional Neural Network

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)
        
        x = torch.randn(IMG_SIZE,IMG_SIZE).view(-1,1,IMG_SIZE,IMG_SIZE)
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 2)
        
    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))        
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))
        
        #print(x[0].shape)
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
net = Net()

### Read the data from file

In [3]:
import numpy as np
training_data = np.load('training_data.npy', allow_pickle=True)
print(len(training_data))

1364


### Split the data up into trainig and test data

In [4]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1, IMG_SIZE, IMG_SIZE)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1
val_size = int(len(X)*VAL_PCT)
print(str(val_size) + ' test samples')

train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

136 test samples


### Train the model (old way)

### Test the model for accuracy by percent and with Heidke Skill Score (old way)

### Calculate loss and accuracy for loging

In [5]:
def fwd_pass(X, y, train=False):
    if train:
        net.zero_grad()
    outputs = net(X)
    matches = [torch.argmax(i) == torch.argmax(j) for i, j in zip(outputs, y)]
    acc = matches.count(True)/len(matches)
    loss = loss_function(outputs, y)
    
    if train:
        loss.backward()
        optimizer.step()
    return acc, loss

### Test function for loging

In [6]:
def test(size=32):
    random_start = np.random.randint(len(test_X)-size)
    X, y = test_X[random_start:random_start+size], test_y[random_start:random_start+size]
    val_acc, val_loss = fwd_pass(X.view(-1,1,IMG_SIZE,IMG_SIZE), y)
    return val_acc, val_loss

### Train function with loging

In [21]:
import time
from tqdm import tqdm

MODEL_NAME = f"model-{int(time.time())}"

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

print(MODEL_NAME)

def train():
    BATCH_SIZE = 100
    EPOCHS = 3
    with open("model.log", "a") as f:
        for epoch in range(EPOCHS):
            for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
                batch_X = train_X[i:i+BATCH_SIZE].view(-1,1,IMG_SIZE,IMG_SIZE)
                batch_y = train_y[i:i+BATCH_SIZE]
                
                acc, loss = fwd_pass(batch_X, batch_y, train=True)
                
                number_of_datapoints_for_one_log = 10
                if i % number_of_datapoints_for_one_log == 0:
                    val_acc, val_loss = test(size=10)
                    f.write(f"{MODEL_NAME}, {round(time.time(), 3)}, {round(float(acc), 2)}, {round(float(loss), 4)}, {round(float(val_acc), 2)}, {round(float(val_loss), 4)}, {epoch}\n")
train()

  0%|          | 0/13 [00:00<?, ?it/s]

model-1585925776


100%|██████████| 13/13 [00:05<00:00,  2.36it/s]


### Graph function

In [70]:
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import style

style.use("ggplot")

def create_acc_loss_graph(model_name):
    contents = open("model.log", "r").read().split('\n')
    
    times = []
    accuracies = []
    losses = []

    val_accs = []
    val_losses = []
    
    epochs = []
    epochs_times = []
    
    for c in contents:
        if model_name in c:
            name, timestamp, acc, loss, val_acc, val_loss, epoch = c.split(",")

            times.append(float(timestamp))
            accuracies.append(float(acc))
            losses.append(float(loss))

            val_accs.append(float(val_acc))
            val_losses.append(float(val_loss))
                        
            if len(epochs) == 0:
                epochs.append(float(epoch))
                epochs_times.append(float(timestamp))
            else:
                if float(epoch) != epochs[-1]:
                    epochs.append(float(epoch))
                    epochs_times.append(float(timestamp))
    
    print(epochs)
    print(epochs_times)

    fig = plt.figure()

    ax1 = plt.subplot2grid((2,1), (0,0))
    ax2 = plt.subplot2grid((2,1), (1,0), sharex=ax1)

    ax1.plot(times, accuracies, label="acc (training data)")
    ax1.plot(times, val_accs, label="val_acc (test data)")
    ax1.plot(epochs_times, epochs, 'ro', label="start epochs")
    ax1.legend(loc=2)

    ax2.plot(times, losses, label="loss (training data)")
    ax2.plot(times, val_losses, label="val_loss (test data)")
    ax2.plot(epochs_times, epochs, 'ro', label="start epochs")
    ax2.legend(loc=2)
    

    plt.show()

create_acc_loss_graph(MODEL_NAME)

[0.0, 1.0, 2.0]
[1585925777.387, 1585925783.018, 1585925788.359]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …